<a href="https://colab.research.google.com/github/karinapinotti/Cobol-GPAZ0801/blob/main/banking-fraud-etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O processo de ETL foi implementado utilizando a biblioteca **Pandas** em **Python**. A etapa de extração consistiu na leitura de dados transacionais bancários a partir de um arquivo CSV. Na transformação, foram aplicadas regras de negócio para identificação de padrões suspeitos, como transações em horários atípicos, valores elevados e múltiplas transações em curto intervalo de tempo. Por fim, os dados tratados foram carregados em um novo arquivo CSV, prontos para análise de fraudes.

In [1]:
import pandas as pd
from datetime import datetime

**EXTRAÇÃO**: Leitura do arquivo CSV de transações bancárias

In [2]:
# Extração dos dados
df = pd.read_csv("transacoes_bancarias.csv")

# Visualizar as primeiras linhas
print(df.head())

   id_transacao  id_cliente       data_transacao    valor moeda  \
0             1        1001  2024-11-01 08:15:00    150.0   BRL   
1             2        1002  2024-11-01 02:45:00   9800.0   BRL   
2             3        1001  2024-11-01 08:17:00    160.0   BRL   
3             4        1003  2024-11-01 23:50:00     50.0   BRL   
4             5        1004  2024-11-01 03:10:00  12000.0   BRL   

  tipo_transacao   canal          cidade estado dispositivo  
0            PIX  Mobile       São Paulo     SP     Android  
1  Transferência     Web  Rio de Janeiro     RJ     Desktop  
2            PIX  Mobile       São Paulo     SP     Android  
3         Cartão     POS  Belo Horizonte     MG         POS  
4  Transferência     Web        Curitiba     PR     Desktop  


**TRANSFORMAÇÃO**

In [3]:
# Conversão de tipos
# Converter data para datetime
df["data_transacao"] = pd.to_datetime(df["data_transacao"])

# Garantir que o valor seja numérico
df["valor"] = pd.to_numeric(df["valor"])


In [4]:
# Criar indicador de transação na madrugada (00h–05h)
df["hora"] = df["data_transacao"].dt.hour

df["transacao_madrugada"] = df["hora"].apply(
    lambda x: 1 if x >= 0 and x <= 5 else 0
)


In [5]:
# Criar indicador de valor alto (> R$ 5.000)
df["valor_alto"] = df["valor"].apply(
    lambda x: 1 if x > 5000 else 0
)


 Identificar múltiplas transações em curto período

 **Critério**:

1.   Mesmo cliente
2.   Intervalo menor que 5 minutos

In [6]:
# Ordenar por cliente e data
df = df.sort_values(by=["id_cliente", "data_transacao"])

# Calcular diferença de tempo entre transações
df["diff_minutos"] = df.groupby("id_cliente")["data_transacao"].diff().dt.total_seconds() / 60

df["muitas_transacoes_seguidas"] = df["diff_minutos"].apply(
    lambda x: 1 if x is not None and x <= 5 else 0
)


Criar flag final de suspeita de fraude

**Regra:**
Se qualquer indicador for verdadeiro → suspeita de fraude

In [7]:
df["suspeita_fraude"] = df.apply(
    lambda row: 1 if (
        row["transacao_madrugada"] == 1 or
        row["valor_alto"] == 1 or
        row["muitas_transacoes_seguidas"] == 1
    ) else 0,
    axis=1
)


In [8]:
# Selecionar colunas finais
df_final = df[
    [
        "id_transacao",
        "id_cliente",
        "data_transacao",
        "valor",
        "tipo_transacao",
        "canal",
        "cidade",
        "estado",
        "transacao_madrugada",
        "valor_alto",
        "muitas_transacoes_seguidas",
        "suspeita_fraude"
    ]
]


In [9]:
# CARGA
df_final.to_csv("fraudes_bancarias_analise.csv", index=False)

print("ETL finalizado com sucesso!")


ETL finalizado com sucesso!
